LUPKER MODEL

In [2]:
import numpy as np
from scipy.optimize import differential_evolution
import math

# Function to read data from external file
def read_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    v_values = np.fromstring(lines[0].split(':')[1], dtype=float, sep=' ')
    P_values = np.fromstring(lines[1].split(':')[1], dtype=float, sep=' ')
    mu_data = np.loadtxt(lines[3:])
    return v_values, P_values, mu_data

# Function to calculate friction coefficient
def friction_coefficient(vt, pn, p, v, p0, k, mu_s, mu_m, u, v_max, h):
    return ((pn / p0) ** -k) * (mu_s + (mu_m - mu_s) * np.exp(-h**2 * (np.log(u / v_max)**2)))

# Function to define objective function
def objective_function(params, vt, pn, mu):
    p, v = params[:2]
    other_params = params[2:]
    mu_calc = friction_coefficient(vt, pn, p, v, *other_params)
    absolute_percentage_error = np.abs((mu_calc - mu) / mu)
    return np.mean(absolute_percentage_error)

# Function to perform optimization
def optimize_parameters(v_values, P_values, mu_data, bounds):
    VT, PN = np.meshgrid(v_values, P_values)
    mu = mu_data.T.ravel()
    result = differential_evolution(objective_function, bounds, args=(VT.flatten(), PN.flatten(), mu),
                                    strategy='best1bin', popsize=20, tol=0.001, mutation=(0.5, 1), recombination=0.7)
    return result.x, result.fun

# Function to get user input for VT and PN
def get_user_input():
    vt_input = float(input("\nEnter the value for VT (mm/s): "))
    pn_input = float(input("Enter the value for PN (MPa): "))
    return vt_input, pn_input

# Function to calculate friction coefficient for user input
def calculate_friction_coefficient(vt_input, pn_input, params):
    calculated_mu_input = friction_coefficient(vt_input, pn_input, *params)
    return calculated_mu_input

# Function to calculate MAPE between calculated and actual mu values
def calculate_mape(calculated_mu_input, actual_mu):
    mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100
    return mape

# Function to print the friction coefficients for all combinations of VT and PN using the optimized parameters
def print_friction_coefficients(v_values, P_values, params):
    print("\nFriction coefficients for all combinations of VT and PN:")
    for i in range(len(v_values)):
        for j in range(len(P_values)):
            calculated_mu = friction_coefficient(v_values[i], P_values[j], *params)
            print("VT={:.1f} mm/s, PN={:.1f} MPa: {:.4f}".format(v_values[i], P_values[j], calculated_mu))

# Function to calculate the overall MAPE between actual and calculated mu values
def calculate_overall_mape(mu_data, v_values, P_values, params):
    calculated_mu_data = np.zeros_like(mu_data)
    for i in range(len(v_values)):
        for j in range(len(P_values)):
            calculated_mu_data[i, j] = friction_coefficient(v_values[i], P_values[j], *params)
    absolute_percentage_error = np.abs((calculated_mu_data - mu_data) / mu_data)
    overall_mape = np.mean(absolute_percentage_error) * 100
    return overall_mape

# Main function
def main():
    # Read data from external file
    v_values, P_values, mu_data = read_data('/content/data.txt')

    # Define bounds for parameters
    bounds = [(0.1, 100.0), (0.1, 100.0), (0.1, 2), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0), (0.1, 10.0)]

    # Perform optimization
    params, mape = optimize_parameters(v_values, P_values, mu_data, bounds)

    # Print the optimal parameter values
    print("\nOptimal parameter values after Differential Evolution optimization:")
    print("p = {:.4f}, v = {:.4f}, p0 = {:.4f}, k = {:.4f}, mu_s = {:.4f}, mu_m = {:.4f}, u = {:.4f}, v_max = {:.4f}, h = {:.4f}".format(*params))

    # Get user input for VT and PN
    vt_input, pn_input = get_user_input()

    # Calculate friction coefficient for user input using optimized parameters
    calculated_mu_input = calculate_friction_coefficient(vt_input, pn_input, params)
    print("\nFriction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

    # Calculate MAPE between calculated and actual mu values
    actual_mu = mu_data[np.abs(v_values - vt_input).argmin(), np.abs(P_values - pn_input).argmin()]
    mape = calculate_mape(calculated_mu_input, actual_mu)

   # Print the friction coefficients for all combinations of VT and PN using the optimized parameters
    print_friction_coefficients(v_values, P_values, params)

    # Calculate and print the overall MAPE between actual and calculated mu values
    overall_mape = calculate_overall_mape(mu_data, v_values, P_values, params)
    print("Overall Mean Absolute Percentage Error (MAPE) between actual and calculated mu values: {:.4f}%".format(overall_mape))

if __name__ == "__main__":
    main()


Optimal parameter values after Differential Evolution optimization:
p = 36.8015, v = 62.2874, p0 = 0.9209, k = 0.2466, mu_s = 0.5986, mu_m = 8.1693, u = 1.7566, v_max = 6.6511, h = 4.0069

Enter the value for VT (mm/s): 1000
Enter the value for PN (MPa): 2.0

Friction coefficient at VT=1000.0 mm/s and PN=2.0 MPa: 0.4944

Friction coefficients for all combinations of VT and PN:
VT=1.0 mm/s, PN=0.1 MPa: 1.0348
VT=1.0 mm/s, PN=0.5 MPa: 0.6959
VT=1.0 mm/s, PN=1.0 MPa: 0.5865
VT=1.0 mm/s, PN=1.5 MPa: 0.5307
VT=1.0 mm/s, PN=2.0 MPa: 0.4944
VT=10.0 mm/s, PN=0.1 MPa: 1.0348
VT=10.0 mm/s, PN=0.5 MPa: 0.6959
VT=10.0 mm/s, PN=1.0 MPa: 0.5865
VT=10.0 mm/s, PN=1.5 MPa: 0.5307
VT=10.0 mm/s, PN=2.0 MPa: 0.4944
VT=100.0 mm/s, PN=0.1 MPa: 1.0348
VT=100.0 mm/s, PN=0.5 MPa: 0.6959
VT=100.0 mm/s, PN=1.0 MPa: 0.5865
VT=100.0 mm/s, PN=1.5 MPa: 0.5307
VT=100.0 mm/s, PN=2.0 MPa: 0.4944
VT=1000.0 mm/s, PN=0.1 MPa: 1.0348
VT=1000.0 mm/s, PN=0.5 MPa: 0.6959
VT=1000.0 mm/s, PN=1.0 MPa: 0.5865
VT=1000.0 mm/s, PN=

WRIGGERS MODEL 2006

In [3]:
import numpy as np
from scipy.optimize import differential_evolution

# Define the mu equation
def mu_equation(vt_pn, c1, c2, c3, c4, c5, c6, c7):
    vt, pn = vt_pn
    return (c1 * ((pn / c2) ** c3) + c4 * np.log(vt / c5) - c6 * np.log(vt / c7))

# Read data from external file
def read_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    v_values = np.fromstring(lines[0].split(':')[1], dtype=float, sep=' ')
    P_values = np.fromstring(lines[1].split(':')[1], dtype=float, sep=' ')
    mu_data = np.loadtxt(lines[3:])
    return v_values, P_values, mu_data

# Define the objective function for differential evolution
def objective_function(params, vt_pn, mu_data):
    mu_calc = mu_equation(vt_pn, *params)
    absolute_percentage_error = np.abs((mu_calc - mu_data) / mu_data)
    return np.mean(absolute_percentage_error)

# Main function
def main():
    # Read data from external file
    v_values, P_values, mu_data = read_data('/content/data.txt')

    # Create meshgrid of VT and PN
    VT, PN = np.meshgrid(v_values, P_values)

    # Prepare the dependent variable (mu)
    mu = mu_data.T.ravel()

    # Define the bounds for parameters
    bounds = [(0.1, 20), (0.1, 20), (-1.0, 1.0), (0.1, 10.0), (0.1, 20), (0.1, 10.0), (0.1, 1.0)]

    # Perform optimization using Differential Evolution
    np.random.seed(42)  # Setting random seed for reproducibility
    result = differential_evolution(objective_function, bounds, args=(np.vstack((VT.flatten(), PN.flatten())), mu), strategy='best1bin', popsize=50, tol=0.01)

    # Extract optimal parameter values
    popt = result.x

    # Print the optimal parameter values
    print("\nOptimal parameter values after Differential Evolution optimization:")
    print("c1 = {:.4f}, c2 = {:.4f}, c3 = {:.4f}, c4 = {:.4f}, c5 = {:.4f}, c6 = {:.4f}, c7 = {:.4f}".format(*popt))

    # Input VT and PN values
    vt_input = float(input("\nEnter the value for VT (mm/s): "))
    pn_input = float(input("Enter the value for PN (MPa): "))

    # Calculate the friction coefficient for the input VT and PN using optimized parameters
    calculated_mu_input = mu_equation((vt_input, pn_input), *popt)
    print("\nFriction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

    # Find the index of the closest VT and PN values in the data
    vt_index = np.argmin(np.abs(v_values - vt_input))
    pn_index = np.argmin(np.abs(P_values - pn_input))

    # Get the corresponding mu value from the data
    actual_mu = mu_data[vt_index, pn_index]

    # Calculate MAPE between calculated and actual mu values
    mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100

    # Print the friction coefficients for all combinations of VT and PN using the optimized parameters
    print("\nFriction coefficients for all combinations of VT and PN:")
    for i in range(len(v_values)):
        for j in range(len(P_values)):
            calculated_mu = mu_equation((v_values[i], P_values[j]), *popt)
            print("VT={:.1f} mm/s, PN={:.1f} MPa: {:.4f}".format(v_values[i], P_values[j], calculated_mu))

    # Calculate the overall MAPE
    mape_array = np.abs((mu_equation((VT.flatten(), PN.flatten()), *popt) - mu_data.T.ravel()) / mu_data.T.ravel()) * 100
    overall_mape = np.mean(mape_array)

    # Print the overall MAPE
    print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points: {:.4f}%".format(overall_mape))

if __name__ == "__main__":
    main()


Optimal parameter values after Differential Evolution optimization:
c1 = 0.7701, c2 = 11.9627, c3 = -0.1367, c4 = 0.1665, c5 = 14.8054, c6 = 0.1357, c7 = 0.3582

Enter the value for VT (mm/s): 1000
Enter the value for PN (MPa): 2.0

Friction coefficient at VT=1000.0 mm/s and PN=2.0 MPa: 0.6082

Friction coefficients for all combinations of VT and PN:
VT=1.0 mm/s, PN=0.1 MPa: 0.8929
VT=1.0 mm/s, PN=0.5 MPa: 0.6005
VT=1.0 mm/s, PN=1.0 MPa: 0.4931
VT=1.0 mm/s, PN=1.5 MPa: 0.4348
VT=1.0 mm/s, PN=2.0 MPa: 0.3954
VT=10.0 mm/s, PN=0.1 MPa: 0.9639
VT=10.0 mm/s, PN=0.5 MPa: 0.6715
VT=10.0 mm/s, PN=1.0 MPa: 0.5640
VT=10.0 mm/s, PN=1.5 MPa: 0.5058
VT=10.0 mm/s, PN=2.0 MPa: 0.4663
VT=100.0 mm/s, PN=0.1 MPa: 1.0348
VT=100.0 mm/s, PN=0.5 MPa: 0.7424
VT=100.0 mm/s, PN=1.0 MPa: 0.6350
VT=100.0 mm/s, PN=1.5 MPa: 0.5767
VT=100.0 mm/s, PN=2.0 MPa: 0.5373
VT=1000.0 mm/s, PN=0.1 MPa: 1.1057
VT=1000.0 mm/s, PN=0.5 MPa: 0.8134
VT=1000.0 mm/s, PN=1.0 MPa: 0.7059
VT=1000.0 mm/s, PN=1.5 MPa: 0.6477
VT=1000.0 m

DORSH MODEL


In [4]:
import numpy as np
from scipy.optimize import differential_evolution

# Define the friction coefficient function
def friction_coefficient(vt_pn, c1, c2, c3):
    vt, pn = vt_pn
    return c1 * (pn ** c2) * (vt ** c3)

# Read data from external file
with open('/content/data.txt', 'r') as file:
    lines = file.readlines()

# Extract v_values, P_values, and mu_data
v_values = np.array([float(val) for val in lines[0].split(':')[1].split()])
P_values = np.array([float(val) for val in lines[1].split(':')[1].split()])
mu_data = np.array([[float(val) for val in line.split()] for line in lines[3:]])

# Create meshgrid of VT and PN
VT, PN = np.meshgrid(v_values, P_values)

# Prepare the dependent variable (mu)
mu = mu_data.T.ravel()

# Define objective function for optimization
def objective_function(params):
    c1, c2, c3 = params
    mu_calc = friction_coefficient((VT.flatten(), PN.flatten()), c1, c2, c3)
    absolute_error = np.abs(mu_calc - mu)
    return np.mean(absolute_error)

# Define bounds for parameters based on the observed range in the table values
bounds = [(0.1, 2), (-2, 2), (-2, 2)]

# Set the initial guess for parameters
initial_guess = np.random.uniform([0.1, -2, -2], [2, 2, 2], size=(10, 3))

# Perform optimization using Differential Evolution with Simulated Annealing
result = differential_evolution(objective_function, bounds, strategy='best1bin', tol=1e-4, seed=42, init=initial_guess)

# Extract optimal parameter values
c1_opt, c2_opt, c3_opt = result.x

# Print the optimal parameter values
print("Optimal parameter values after Differential Evolution optimization with Simulated Annealing:")
print("C1 = {:.4f}, C2 = {:.4f}, C3 = {:.4f}".format(c1_opt, c2_opt, c3_opt))

# Input VT and PN values
vt_input = float(input("\nEnter the value for VT (mm/s): "))
pn_input = float(input("Enter the value for PN (MPa): "))

# Calculate the friction coefficient for the input VT and PN using optimized parameters
calculated_mu_input = friction_coefficient((vt_input, pn_input), c1_opt, c2_opt, c3_opt)
print("\nFriction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

# Find the index of the closest VT and PN values in the data
vt_index = np.argmin(np.abs(v_values - vt_input))
pn_index = np.argmin(np.abs(P_values - pn_input))

# Get the corresponding mu value from the data
actual_mu = mu_data[vt_index, pn_index]

# Calculate MAPE between calculated and actual mu values
mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100


# Print all mu values for respective VT and PN
print("\nCalculated mu values for all combinations of VT and PN:")
for i, vt in enumerate(v_values):
    for j, pn in enumerate(P_values):
        mu_calc = friction_coefficient((vt, pn), c1_opt, c2_opt, c3_opt)
        print("VT={:.1f} mm/s, PN={:.1f} MPa: Calculated Mu = {:.4f}".format(vt, pn, mu_calc))

# Calculate MAPE between calculated and actual mu values for all data points
mape_array = np.abs((friction_coefficient((VT.flatten(), PN.flatten()), c1_opt, c2_opt, c3_opt) - mu_data.T.ravel()) / mu_data.T.ravel()) * 100
overall_mape = np.mean(mape_array)

# Print the overall MAPE
print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points: {:.4f}%".format(overall_mape))

Optimal parameter values after Differential Evolution optimization with Simulated Annealing:
C1 = 0.4934, C2 = -0.2660, C3 = 0.0543

Enter the value for VT (mm/s): 1000
Enter the value for PN (MPa): 2.0

Friction coefficient at VT=1000.0 mm/s and PN=2.0 MPa: 0.5971

Calculated mu values for all combinations of VT and PN:
VT=1.0 mm/s, PN=0.1 MPa: Calculated Mu = 0.9104
VT=1.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.5933
VT=1.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.4934
VT=1.0 mm/s, PN=1.5 MPa: Calculated Mu = 0.4429
VT=1.0 mm/s, PN=2.0 MPa: Calculated Mu = 0.4103
VT=10.0 mm/s, PN=0.1 MPa: Calculated Mu = 1.0317
VT=10.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.6723
VT=10.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.5591
VT=10.0 mm/s, PN=1.5 MPa: Calculated Mu = 0.5019
VT=10.0 mm/s, PN=2.0 MPa: Calculated Mu = 0.4650
VT=100.0 mm/s, PN=0.1 MPa: Calculated Mu = 1.1691
VT=100.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.7619
VT=100.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.6336
VT=100.0 mm/s, PN=1.5 MPa: Calculated Mu =

HEUMER MODEL

In [5]:
#DIFFERENTIAL EVOLUTION
import numpy as np
from scipy.optimize import minimize, differential_evolution

# Define the function to fit
def huemer_friction_law(vt_pn, a, b, c, n, m, alpha, beta):
    vt, pn = vt_pn
    return (alpha * np.abs(pn)**(n-1) + beta) / (a + b/(np.abs(vt)**(1/m)) + c/(np.abs(vt)**(2/m)))

# Read data from external file
with open('/content/data.txt', 'r') as file:
    lines = file.readlines()

# Extract v_values, P_values, and mu_data
v_values = np.array([float(val) for val in lines[0].split(':')[1].split()])
P_values = np.array([float(val) for val in lines[1].split(':')[1].split()])
mu_data = np.array([[float(val) for val in line.split()] for line in lines[3:]])

# Create meshgrid of VT and PN
VT, PN = np.meshgrid(v_values, P_values)

# Prepare the dependent variable (mu)
mu = mu_data.T.ravel()

# Define objective function for optimization
def objective_function(params):
    a, b, c, n, m, alpha, beta = params
    mu_calc = huemer_friction_law((VT.flatten(), PN.flatten()), a, b, c, n, m, alpha, beta)
    absolute_percentage_error = np.abs((mu_calc - mu) / mu) * 100
    return np.mean(absolute_percentage_error)

# Define bounds for parameters
bounds = [(1.0, 10.0), (-10.0, 1.0), (1.0, 10.0), (1.0, 10.0), (1.0, 10.0), (-10.0, -1.0), (1.0, 10.0)]
# Set random seed for reproducibility
np.random.seed(42)
# Perform optimization using Differential Evolution with bounds and strategy 'best1bin'
result_de = differential_evolution(objective_function, bounds, strategy='best1bin', popsize=30, tol=1e-4)
# Use optimized parameters from Differential Evolution
popt_de = result_de.x
# Perform optimization using L-BFGS-B method with bounds
result_lbfgsb = minimize(objective_function, x0=popt_de, method='L-BFGS-B', bounds=bounds, tol=1e-6)
# Use optimized parameters from L-BFGS-B method
popt_lbfgsb = result_lbfgsb.x
# Print the optimal parameters
print("Optimal parameters after optimization:")
print("a = {:.4f}, b = {:.4f}, c = {:.4f}, n = {:.4f}, m = {:.4f}, alpha = {:.4f}, beta = {:.4f}".format(*popt_lbfgsb))

# Input VT and PN values
vt_input = float(input("Enter the value for VT (mm/s): "))
pn_input = float(input("Enter the value for PN (MPa): "))

# Calculate the friction coefficient for the input VT and PN using optimized parameters from L-BFGS-B method
calculated_mu_input = huemer_friction_law((vt_input, pn_input), *popt_lbfgsb)
print("Friction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

# Find the index of the closest VT and PN values in the data
vt_index = np.argmin(np.abs(v_values - vt_input))
pn_index = np.argmin(np.abs(P_values - pn_input))

# Get the corresponding mu value from the data
actual_mu = mu_data[vt_index, pn_index]

# Calculate MAPE between calculated and actual mu values
mape = np.abs((calculated_mu_input - actual_mu) / actual_mu) * 100

# Print all mu values for respective VT and PN
print("\nCalculated mu values for all combinations of VT and PN:")
for i, vt in enumerate(v_values):
    for j, pn in enumerate(P_values):
        mu_calc = huemer_friction_law((vt, pn), *popt_lbfgsb)
        print("VT={:.1f} mm/s, PN={:.1f} MPa: Calculated Mu = {:.4f}".format(vt, pn, mu_calc))

# Calculate MAPE between calculated and actual mu values for all data points
mape_array = np.abs((huemer_friction_law((VT.flatten(), PN.flatten()), *popt_lbfgsb) - mu_data.T.ravel()) / mu_data.T.ravel()) * 100
overall_mape = np.mean(mape_array)

# Print the overall MAPE
print("\nOverall Mean Absolute Percentage Error (MAPE) for all data points: {:.4f}%".format(overall_mape))





Optimal parameters after optimization:
a = 1.0042, b = -1.1221, c = 1.2305, n = 1.0312, m = 9.9956, alpha = -5.1192, beta = 5.6492
Enter the value for VT (mm/s): 1000
Enter the value for PN (MPa): 2.0
Friction coefficient at VT=1000.0 mm/s and PN=2.0 MPa: 0.5569

Calculated mu values for all combinations of VT and PN:
VT=1.0 mm/s, PN=0.1 MPa: Calculated Mu = 0.7951
VT=1.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.5747
VT=1.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.4764
VT=1.0 mm/s, PN=1.5 MPa: Calculated Mu = 0.4178
VT=1.0 mm/s, PN=2.0 MPa: Calculated Mu = 0.3759
VT=10.0 mm/s, PN=0.1 MPa: Calculated Mu = 0.9949
VT=10.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.7191
VT=10.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.5960
VT=10.0 mm/s, PN=1.5 MPa: Calculated Mu = 0.5228
VT=10.0 mm/s, PN=2.0 MPa: Calculated Mu = 0.4703
VT=100.0 mm/s, PN=0.1 MPa: Calculated Mu = 1.1255
VT=100.0 mm/s, PN=0.5 MPa: Calculated Mu = 0.8135
VT=100.0 mm/s, PN=1.0 MPa: Calculated Mu = 0.6743
VT=100.0 mm/s, PN=1.5 MPa: Calculated Mu = 0.

WRIGGERS AND REINELT


In [7]:
import numpy as np
from scipy.optimize import differential_evolution

# Function to read data from external file
def read_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    v_values = np.fromstring(lines[0].split(':')[1], dtype=float, sep=' ')
    P_values = np.fromstring(lines[1].split(':')[1], dtype=float, sep=' ')
    mu_data = np.loadtxt(lines[3:])
    return v_values, P_values, mu_data

# Function to calculate friction coefficient
def friction_coefficient(v, p, a, b, c, d):
    mu_max = (2 * v * a * p / (v**2 + (a * p)**2))**c * (b / p) * np.arctan(d * p)
    return mu_max

# Function to define objective function
def objective_function(params, vt, pn, mu):
    a, b, c, d = params
    mu_calc = friction_coefficient(vt, pn, a, b, c, d)
    absolute_percentage_error = np.abs((mu_calc - mu) / mu)
    return np.mean(absolute_percentage_error)

def optimize_parameters(v_values, P_values, mu_data):
    bounds = [(0, 1), (0, 1), (-1, 0), (1, 2)]  # Tighter bounds
    VT, PN = np.meshgrid(v_values, P_values)
    mu = mu_data.T.ravel()
    result = differential_evolution(objective_function, bounds, args=(VT.flatten(), PN.flatten(), mu),
                                    strategy='best1bin', popsize=50, tol=0.0001, mutation=(0.5, 1.9), recombination=0.7)
    return result.x, result.fun

# Function to get user input for VT and PN
def get_user_input(v_values, P_values):
    vt_input = float(input("\nEnter the value for VT (mm/s): "))
    pn_input = float(input("Enter the value for PN (MPa): "))
    return vt_input, pn_input

# Function to calculate friction coefficient for user input using optimized parameters
def calculate_friction_coefficient(vt_input, pn_input, params):
    calculated_mu_input = friction_coefficient(vt_input, pn_input, *params)
    return calculated_mu_input

# Function to calculate the overall MAPE between actual and calculated mu values
def calculate_overall_mape(mu_data, v_values, P_values, params):
    calculated_mu_data = np.zeros_like(mu_data)
    for i in range(len(v_values)):
        for j in range(len(P_values)):
            calculated_mu_data[i, j] = friction_coefficient(v_values[i], P_values[j], *params)
    absolute_percentage_error = np.abs((calculated_mu_data - mu_data) / mu_data)
    overall_mape = np.mean(absolute_percentage_error) * 100
    return overall_mape

# Main function
def main():
    # Read data from external file
    v_values, P_values, mu_data = read_data('/content/data.txt')

    # Perform optimization
    params, mape = optimize_parameters(v_values, P_values, mu_data)

    # Print the optimal parameter values
    print("\nOptimal parameter values after Differential Evolution optimization:")
    print("a = {:.4f}, b = {:.4f}, c = {:.4f}, d = {:.4f}".format(*params))

    # Get user input for VT and PN
    vt_input, pn_input = get_user_input(v_values, P_values)

    # Calculate friction coefficient for user input using optimized parameters
    calculated_mu_input = calculate_friction_coefficient(vt_input, pn_input, params)
    print("\nFriction coefficient at VT={:.1f} mm/s and PN={:.1f} MPa: {:.4f}".format(vt_input, pn_input, calculated_mu_input))

    # Print all the corresponding mu values for (VT and Pn) from the input table
    print("\nCorresponding mu values for all combinations of VT and PN:")
    for i in range(len(v_values)):
        for j in range(len(P_values)):
            calculated_mu = friction_coefficient(v_values[i], P_values[j], *params)
            print("VT={:.1f} mm/s, PN={:.1f} MPa: {:.4f}".format(v_values[i], P_values[j], calculated_mu))

    # Calculate and print the overall MAPE between actual and calculated mu values
    overall_mape = calculate_overall_mape(mu_data, v_values, P_values, params)
    print("\nOverall Mean Absolute Percentage Error (MAPE) between actual and calculated mu values: {:.4f}%".format(overall_mape))

if __name__ == "__main__":
    main()


Optimal parameter values after Differential Evolution optimization:
a = 0.2399, b = 0.5915, c = -0.0755, d = 1.0000

Enter the value for VT (mm/s): 1000
Enter the value for PN (MPa): 2.0

Friction coefficient at VT=1000.0 mm/s and PN=2.0 MPa: 0.5534

Corresponding mu values for all combinations of VT and PN:
VT=1.0 mm/s, PN=0.1 MPa: 0.7415
VT=1.0 mm/s, PN=0.5 MPa: 0.6115
VT=1.0 mm/s, PN=1.0 MPa: 0.4931
VT=1.0 mm/s, PN=1.5 MPa: 0.4009
VT=1.0 mm/s, PN=2.0 MPa: 0.3336
VT=10.0 mm/s, PN=0.1 MPa: 0.8823
VT=10.0 mm/s, PN=0.5 MPa: 0.7269
VT=10.0 mm/s, PN=1.0 MPa: 0.5843
VT=10.0 mm/s, PN=1.5 MPa: 0.4728
VT=10.0 mm/s, PN=2.0 MPa: 0.3909
VT=100.0 mm/s, PN=0.1 MPa: 1.0499
VT=100.0 mm/s, PN=0.5 MPa: 0.8650
VT=100.0 mm/s, PN=1.0 MPa: 0.6952
VT=100.0 mm/s, PN=1.5 MPa: 0.5625
VT=100.0 mm/s, PN=2.0 MPa: 0.4650
VT=1000.0 mm/s, PN=0.1 MPa: 1.2492
VT=1000.0 mm/s, PN=0.5 MPa: 1.0293
VT=1000.0 mm/s, PN=1.0 MPa: 0.8273
VT=1000.0 mm/s, PN=1.5 MPa: 0.6693
VT=1000.0 mm/s, PN=2.0 MPa: 0.5534

Overall Mean Absol